In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import annotations

In [4]:
import numpy as np
from qat.model.component import Component, ComponentId, make_refdict
from qat.model.refs import Ref, RefDict, RefList
from qat.model.autopopulate import AutoPopulate

In [5]:
class At(Component):
    x: int
    
class Bt(Component):
    x: int
    s: str
    As: RefDict[At]
    Ds: RefList[Dt]

class Ct(Component):
    x: int
    As: RefDict[At]
    Bs: RefList[Bt]
    someB: Ref[Bt]

class Dt(Component):
    x: int
    Cs: RefDict[Ct]

In [6]:
class Outer(AutoPopulate):
    A: list[At]
    B: list[Bt]
    C: list[Ct]
    D: list[Dt]

In [7]:
def make_Outer(count=10, connections=3, seed=42):
    rng = np.random.default_rng(seed)
    pick = lambda L, size=3: make_refdict(*rng.choice(L, size=size))
    A = [At(x=rng.integers(100)) for _ in range(count)]
    B = [
        Bt(x=rng.integers(100), As=pick(A, 3), s=f"blah{rng.integers(100)}", Ds=[])
        for i in range(count)
    ]
    C = [
        Ct(
            x=rng.integers(100),
            As=pick(A, 3),
            Bs=list(pick(B, 3).values()),
            someB=list(pick(B, 1).values())[0],
        )
        for i in range(count)
    ]
    D = [Dt(x=rng.integers(100), Cs=pick(C, 3)) for i in range(count)]

    for bt in B:
        bt.Ds = list(pick(D, 3).values())

    return Outer(A=A, B=B, C=C, D=D, id="outer")


In [8]:
O1 = make_Outer()

In [9]:
print(O1.model_dump_json(indent=2))

{
  "A": [
    {
      "uuid": "0052fa4c-9272-417f-91f7-180baa876d14",
      "id_type": "At",
      "x": 8
    },
    {
      "uuid": "51c7a48c-58fb-4122-8268-61a1f3addcd4",
      "id_type": "At",
      "x": 77
    },
    {
      "uuid": "77c0fb82-06ae-46b1-8343-84d9e369c055",
      "id_type": "At",
      "x": 65
    },
    {
      "uuid": "1a750171-6626-44d0-a709-2b647f7e70dd",
      "id_type": "At",
      "x": 43
    },
    {
      "uuid": "b043d1ea-7257-424b-9ad1-3e797278180f",
      "id_type": "At",
      "x": 43
    },
    {
      "uuid": "87be2f4c-a78c-47dc-9eec-8b7be3f8aaaa",
      "id_type": "At",
      "x": 85
    },
    {
      "uuid": "2565eab4-4e67-4c0d-aba7-97201d2313d9",
      "id_type": "At",
      "x": 8
    },
    {
      "uuid": "c24a210a-45e8-4586-8661-12a46e474f56",
      "id_type": "At",
      "x": 69
    },
    {
      "uuid": "23c142ce-2749-4bd1-9e5e-33af5c84f462",
      "id_type": "At",
      "x": 20
    },
    {
      "uuid": "598351f1-ad3b-4b6a-a9a6-9c3d1c5ccc

In [10]:
O2 = Outer(**O1.model_dump())
O2._deepequals(O1)

True

In [11]:
%%timeit
O1 == O2

8.63 μs ± 97.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [37]:
%%timeit
blob = O1.model_dump()
O2 = Outer(**blob)
blob2 = O2.model_dump()

assert blob == blob2

2.54 ms ± 36.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
